# MLB Salary Prediction - Data Wrangling

There are two datasets that needed to be processed: 1) The salary data: `mlb-free-agency.csv` and 2) The production stats: `mlb-batting.csv`. The salary data are retrieved from [spotrac.com](https://www.spotrac.com/) using my own scraper. They contain MLB free agent transaction information from the 2011 season to 2022 season. The production stats are retreived from [FanGraphs](https://www.fangraphs.com/) using the `pybaseball` package.

## Step 1: Initial processing of salary data

In [1]:
from pybaseball import batting_stats
from pybaseball import playerid_lookup
from pybaseball import player_search_list
from pybaseball import playerid_reverse_lookup
import numpy as np
import pandas as pd
import csv
pd.set_option('display.max_rows', 10)

In [2]:
# import and inspect salary data

salary_df = pd.read_csv('mlb-free-agency.csv')
print(salary_df.shape)
salary_df.head()

(1669, 12)


,Unnamed: 0,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink
0,0,Wil Nieves,C,33.2,WSH,MIL,1,775000.0,775000.0,2011,5414,https://www.spotrac.com/redirect/player/5414/
1,1,Albert Pujols,DH,31.8,STL,LAA,10,240000000.0,24000000.0,2012,795,https://www.spotrac.com/redirect/player/795/
2,2,Prince Fielder,DH,27.7,MIL,DET,9,214000000.0,23777778.0,2012,493,https://www.spotrac.com/redirect/player/493/
3,3,Jose Reyes,SS,28.4,NYM,MIA,6,106000000.0,17666667.0,2012,559,https://www.spotrac.com/redirect/player/559/
4,4,C.J. Wilson,SP,31.0,TEX,LAA,5,77500000.0,15500000.0,2012,874,https://www.spotrac.com/redirect/player/874/


In [3]:
salary_df.info()
# we have lots of entries that are missing salary info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1669 non-null   int64  
 1   name             1669 non-null   object 
 2   position         1669 non-null   object 
 3   age              1669 non-null   float64
 4   from_team        1669 non-null   object 
 5   to_tam           1668 non-null   object 
 6   contract_length  1669 non-null   object 
 7   total_salary     1275 non-null   float64
 8   avg_salary       1275 non-null   float64
 9   year             1669 non-null   int64  
 10  spotracID        1669 non-null   int64  
 11  spotracLink      1669 non-null   object 
dtypes: float64(3), int64(3), object(6)
memory usage: 156.6+ KB


In [4]:
# drop all rows with missing salary value

salary_df = salary_df[salary_df['total_salary'].notna()]
print(f"Total rows left: {len(salary_df.index)}")

Total rows left: 1275


In [5]:
# drop the only entry from year 2011

salary_df = salary_df.drop([0])
# drop first column
salary_df.drop(columns=salary_df.columns[0], axis=1, inplace=True)

In [6]:
# drop all pitchers: SP and RP

drop_SP = salary_df[salary_df['position'] == 'SP'].index
salary_df.drop(drop_SP, inplace = True)
drop_RP = salary_df[salary_df['position'] == 'RP'].index
salary_df.drop(drop_RP, inplace = True)
# change all outfield positions into 'OF'
salary_df.position.replace(['LF', 'CF', 'RF'], 'OF', inplace=True)

In [7]:
salary_df.sort_values('name', inplace=True)
salary_df.reset_index(drop=True, inplace=True)
salary_df.head()

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink
0,A.J. Ellis,C,35.6,PHI,MIA,1,2500000.0,2500000.0,2017,7427,https://www.spotrac.com/redirect/player/7427/
1,A.J. Pierzynski,C,37.5,BOS,STL,1,500000.0,500000.0,2014,188,https://www.spotrac.com/redirect/player/188/
2,A.J. Pierzynski,C,35.9,CHW,TEX,1,7500000.0,7500000.0,2013,188,https://www.spotrac.com/redirect/player/188/
3,A.J. Pierzynski,C,37.9,STL,ATL,1,2000000.0,2000000.0,2015,188,https://www.spotrac.com/redirect/player/188/
4,A.J. Pierzynski,C,38.8,ATL,ATL,1,3000000.0,3000000.0,2016,188,https://www.spotrac.com/redirect/player/188/


In [8]:
# see if more than one players have the same name in our salary data

## make a df with unique spotracIDs
unique = salary_df.drop_duplicates(subset=['spotracID'])
## check if there's any duplicate names in our unique DF
print("Players with the same name:")
unique[unique.duplicated(subset=['name'], keep=False)]
# there isn't any

Players with the same name:


,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink


## Step 2: Assign an ID to each unique player entry
We will need to merge the salary data with the production stats later on in order to perform analysis. Since the salary data came with only Spotrac IDs, we need to somehow lookup every individual player's FanGraphs ID instead, and assign it back to them. Luckily, `pybaseball` is embedded with the `playerid_lookup` function which does just that.

In [9]:
# create a new DataFrame just for the search operation

player_search = salary_df[['name','spotracID','spotracLink']].copy()
player_search.drop_duplicates(subset=['name'], keep='first', inplace=True)
player_search.head()

,name,spotracID,spotracLink
0,A.J. Ellis,7427,https://www.spotrac.com/redirect/player/7427/
1,A.J. Pierzynski,188,https://www.spotrac.com/redirect/player/188/
5,A.J. Pollock,10693,https://www.spotrac.com/redirect/player/10693/
6,Aaron Hill,908,https://www.spotrac.com/redirect/player/908/
7,Abraham Almonte,14343,https://www.spotrac.com/redirect/player/14343/


In [10]:
# split full names into first and last

for i, row in player_search.iterrows():
    player_search.at[i, 'first_name'] = row['name'].split()[0].lower()
    if len(row['name'].split()) <= 2:
        player_search.at[i, 'last_name'] = row['name'].split()[-1].lower()
    else:
        player_search.at[i, 'last_name'] = row['name'].split()[1].lower()
player_search.head()

,name,spotracID,spotracLink,first_name,last_name
0,A.J. Ellis,7427,https://www.spotrac.com/redirect/player/7427/,a.j.,ellis
1,A.J. Pierzynski,188,https://www.spotrac.com/redirect/player/188/,a.j.,pierzynski
5,A.J. Pollock,10693,https://www.spotrac.com/redirect/player/10693/,a.j.,pollock
6,Aaron Hill,908,https://www.spotrac.com/redirect/player/908/,aaron,hill
7,Abraham Almonte,14343,https://www.spotrac.com/redirect/player/14343/,abraham,almonte


In [11]:
# add an empty space into the abbreviated first names: e.g. 'c.j.' into 'c. j.'

for i, row in player_search.iterrows():
    if '.' in row.first_name:
        row.first_name = row.first_name.replace('.', '. ', 1)
        player_search.at[i, 'first_name'] = row.first_name
player_search.loc[[0]]
# Don't run more than once!

,name,spotracID,spotracLink,first_name,last_name
0,A.J. Ellis,7427,https://www.spotrac.com/redirect/player/7427/,a. j.,ellis


In [12]:
# create a list of tuples(last, first) to pass into the playerid_lookup function

name_list = []
for i, row in player_search.iterrows():
    tup = (row.last_name, row.first_name)
    name_list.append(tup)
print(f"# of players to be searched: {len(name_list)}")
name_list[:5]

# of players to be searched: 383


[('ellis', 'a. j.'),
 ('pierzynski', 'a. j.'),
 ('pollock', 'a. j.'),
 ('hill', 'aaron'),
 ('almonte', 'abraham')]

In [13]:
# lookup player IDs

playerID = player_search_list(name_list)
playerID.head()

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,ellis,a. j.,454560,ellia001,ellisaj01,5677,2008.0,2018.0
1,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
2,hill,aaron,431094,hilla001,hillaa01,6104,2005.0,2017.0
3,almonte,abraham,501659,almoa001,almonab01,5486,2013.0,2021.0
4,duvall,adam,594807,duvaa001,duvalad01,10950,2014.0,2021.0


In [14]:
# check if we have any duplicate results

playerID[playerID.duplicated(subset=['name_last', 'name_first'], keep=False)]
# indeed we have a lot

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
17,gonzalez,alex,136460,gonza002,gonzaal02,520,1998.0,2014.0
18,gonzalez,alex,114924,gonza001,gonzaal01,281,1994.0,2006.0
36,hall,bill,115355,hallb107,hallbi01,1005234,1913.0,1913.0
37,hall,bill,115356,hallb105,hallbi02,1005235,1954.0,1958.0
38,hall,bill,407849,hallb001,hallbi03,1605,2002.0,2012.0
...,...,...,...,...,...,...,...,...
288,taylor,michael,446345,taylm001,taylomi01,2591,2011.0,2014.0
301,cruz,nelson,112906,cruzn001,cruzne01,554,1997.0,2003.0
302,cruz,nelson,443558,cruzn002,cruzne02,2434,2005.0,2021.0
320,hernandez,ramon,115831,hernr102,hernara01,1005710,1967.0,1977.0


In [15]:
# remove all players whose retired before 2011

idx_drop = playerID[playerID['mlb_played_last']<=2010].index
playerID.drop(idx_drop, inplace=True)
# check if there are still duplicated players
playerID[playerID.duplicated(subset=['name_last', 'name_first'], keep=False)]

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
84,young,chris,432934,younc003,youngch03,3196,2004.0,2017.0
85,young,chris,455759,younc004,youngch04,3882,2006.0,2018.0
287,taylor,michael,572191,taylm002,taylomi02,11489,2014.0,2021.0
288,taylor,michael,446345,taylm001,taylomi01,2591,2011.0,2014.0


In [16]:
# after looking them up manualy, it turned out that chris young 3196 and taylor michael 2591 
# are not the ones I'm after

cy_idx = playerID[playerID['key_fangraphs']==3196].index
mt_idx = playerID[playerID['key_fangraphs']==2591].index
playerID.drop(cy_idx, inplace=True)
playerID.drop(mt_idx, inplace=True)

In [17]:
# see if there's still duplicate entries

playerID[playerID.duplicated(subset=['name_last', 'name_first'], keep=False)]
# nope

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last


In [18]:
print(f"Number of players we searched: {len(name_list)}")
print(f"Number of result that we got: {len(playerID.index)}")

Number of players we searched: 383
Number of result that we got: 368


In [19]:
# create a list of player name that returned positive results
name_list_positive = []
for i, row in playerID.iterrows():
    tup = (row.name_last, row.name_first)
    name_list_positive.append(tup)
# check who are we still missing
name_list_missing = [x for x in name_list if x not in name_list_positive]
name_list_missing

[('pollock', 'a. j.'),
 ('de', 'alejandro'),
 ('ramirez', 'alexei'),
 ('lemahieu', 'd. j.'),
 ('santana', 'daniel'),
 ('iglesias', 'jose'),
 ('martin', 'leonys'),
 ('pina', 'manuel'),
 ('upton', 'melvin'),
 ('aoki', 'norichika'),
 ('pearce', 'steven'),
 ('la', 'tommy'),
 ('cespedes', 'yoenis'),
 ('torrealba', 'yorbit'),
 ('tsutsugo', 'yoshitomo')]

As you can see, we still ended up with 15 missing players. They were missed due to mismatching name format across databases. I decided to leave them out.

## Step 3: Merge salary data and playerID
Since neither the salary data nor playerID table contains duplicate names, we can safely merge the two on player names, which is the only unique identifier we have anyway. We need to create a `full_name` column in both tables.

In [20]:
# create a 'full_name' column in salary_df

salary_df['full_name'] = salary_df['name'].str.lower().str.replace(' ', '')
salary_df.head()

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink,full_name
0,A.J. Ellis,C,35.6,PHI,MIA,1,2500000.0,2500000.0,2017,7427,https://www.spotrac.com/redirect/player/7427/,a.j.ellis
1,A.J. Pierzynski,C,37.5,BOS,STL,1,500000.0,500000.0,2014,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski
2,A.J. Pierzynski,C,35.9,CHW,TEX,1,7500000.0,7500000.0,2013,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski
3,A.J. Pierzynski,C,37.9,STL,ATL,1,2000000.0,2000000.0,2015,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski
4,A.J. Pierzynski,C,38.8,ATL,ATL,1,3000000.0,3000000.0,2016,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski


In [21]:
# create a 'full_name' column in playerID

playerID['full_name'] = playerID['name_first'] + playerID['name_last']
playerID['full_name'] = playerID['full_name'].str.lower().str.replace(' ', '')
playerID.head()

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,full_name
0,ellis,a. j.,454560,ellia001,ellisaj01,5677,2008.0,2018.0,a.j.ellis
1,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0,a.j.pierzynski
2,hill,aaron,431094,hilla001,hillaa01,6104,2005.0,2017.0,aaronhill
3,almonte,abraham,501659,almoa001,almonab01,5486,2013.0,2021.0,abrahamalmonte
4,duvall,adam,594807,duvaa001,duvalad01,10950,2014.0,2021.0,adamduvall


In [22]:
# perform the merge

salaryFinalDF = pd.merge(salary_df, playerID, on='full_name', how='inner')
salaryFinalDF.head()

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink,full_name,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,A.J. Ellis,C,35.6,PHI,MIA,1,2500000.0,2500000.0,2017,7427,https://www.spotrac.com/redirect/player/7427/,a.j.ellis,ellis,a. j.,454560,ellia001,ellisaj01,5677,2008.0,2018.0
1,A.J. Pierzynski,C,37.5,BOS,STL,1,500000.0,500000.0,2014,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
2,A.J. Pierzynski,C,35.9,CHW,TEX,1,7500000.0,7500000.0,2013,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
3,A.J. Pierzynski,C,37.9,STL,ATL,1,2000000.0,2000000.0,2015,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
4,A.J. Pierzynski,C,38.8,ATL,ATL,1,3000000.0,3000000.0,2016,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0


In [23]:
# check how many rows we have, should be a little over 15

print(f"# of rows before merge: {len(salary_df.index)}")
print(f"# of rows after merge: {len(salaryFinalDF.index)}")

# of rows before merge: 583
# of rows after merge: 561


We lost 22, since some of the 15 missed players had 1+ rows of salary information

In [24]:
# drop the columns we don't need

salaryFinalDF.drop(columns=['contract_length', 
                            'total_salary', 
                            'full_name', 
                            'name_last', 
                            'name_first',
                            'key_mlbam', 
                            'key_retro',
                            'key_bbref',
                            'spotracID',
                            'spotracLink'], inplace=True)
# rename the columns
salaryFinalDF.rename(columns={'year': 'year_fa',
                              'key_fangraphs': 'IDfg',
                              'mlb_played_first': 'first_played',
                              'mlb_played_last': 'last_played',
                              'to_tam': 'to_team'}, inplace=True)
salaryFinalDF.head()

,name,position,age,from_team,to_team,avg_salary,year_fa,IDfg,first_played,last_played
0,A.J. Ellis,C,35.6,PHI,MIA,2500000.0,2017,5677,2008.0,2018.0
1,A.J. Pierzynski,C,37.5,BOS,STL,500000.0,2014,746,1998.0,2016.0
2,A.J. Pierzynski,C,35.9,CHW,TEX,7500000.0,2013,746,1998.0,2016.0
3,A.J. Pierzynski,C,37.9,STL,ATL,2000000.0,2015,746,1998.0,2016.0
4,A.J. Pierzynski,C,38.8,ATL,ATL,3000000.0,2016,746,1998.0,2016.0


## Step 4: Import and clean the batting stats
1. Remove entries with BA less than 400:
2. Remove unwanted variables

In [25]:
# import mlb all batting stats from 2007-2021
batting_df = pd.read_csv('mlb-batting.csv')
pd.set_option('display.max_columns', None)
print(batting_df.shape)
batting_df.head()

# drop rows with AB < 400
# batting_df = batting_df.drop(batting_df[batting_df['AB']<400].index)
# batting_df.shape

(14118, 320)


,Unnamed: 0,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,292,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,47,5,32,129,80,81,8,91,8,5,0,5,30,6,0.346,147,195,92,18,2582,1095,1487,14,0,0,0.132,0.148,0.89,0.438,0.640,1.078,0.294,0.368,0.0075,0.212,0.339,0.449,0.092,0.164,0.095,0.0,0.449,67.2,139,62.4,16.8,18.4,-5.3,101.1,10.4,$83.3,7.0,185,5.77,-8.16,13.93,62.93,6.11,0.92,3.62,4,6.44,-0.15,0.547,93.2,0.199,84.5,0.050,88.7,0.073,79.1,0.114,84.4,0.017,85.4,NaN,NaN,0.009,NaN,40.1,11.2,4.2,1.1,11.1,2.3,NaN,2.84,2.19,3.28,0.58,3.79,5.06,NaN,0.198,0.570,0.356,0.706,0.930,0.859,0.426,0.562,0.050,6.9,0.350,0.097,0.047,0.022,NaN,0.101,0.204,0.058,0.015,NaN,0.106,NaN,NaN,NaN,93.5,92.8,88.4,85.2,NaN,92.1,84.6,78.4,80.8,NaN,84.4,NaN,NaN,-2.4,-6.1,0.4,-4.1,NaN,-1.6,1.6,2.9,4.5,NaN,0.3,NaN,NaN,9.6,6.0,4.9,3.2,NaN,5.4,1.2,-5.3,-5.8,NaN,4.5,NaN,NaN,18.3,13.3,3.1,2.5,NaN,7.1,9.8,1.5,2.5,NaN,10.1,NaN,NaN,2.03,5.33,2.58,4.34,NaN,2.72,1.87,1.03,6.59,NaN,3.72,NaN,NaN,0.158,0.546,0.355,0.635,0.919,0.857,0.509,NaN,11.6,3.1,2.7,25 - 25,69.3,1.9,1.1,0.472,0.348,0.180,0.122,0.433,0.445,0.332,0.105,NaN,0.070,0.366,0.046,0.026,NaN,NaN,0.173,0.201,0.0,84.6,NaN,79.0,93.6,88.6,84.6,NaN,NaN,92.6,84.7,93.5,0.4,NaN,3.6,-1.9,0.5,-3.2,NaN,NaN,-4.2,2.1,-6.4,3.1,NaN,-7.2,8.2,4.2,1.6,NaN,NaN,4.4,-0.2,5.9,10.5,NaN,0.9,18.2,4.0,4.0,NaN,NaN,21.4,10.5,0.0,3.82,NaN,0.48,1.91,3.30,5.95,NaN,NaN,4.75,2.00,2.04,0.162,0.539,0.355,0.647,0.917,0.857,0.512,NaN,NaN,139.0,159,68,137.0,154.0,176.0,125.0,1.00,80.0,123.0,129.0,115.0,102.0,73.0,69.0,92.0,125.0,92.3,18.5,57.0,0.131,110.6,217.0,0.5,434,0.220,0.270,NaN,NaN,NaN
1,489,10155,2013,Mike Trout,LAA,21,157,589,716,190

In [26]:
# drop all unwanted stats

var_list_basic = ['IDfg', 
            'Season', 
            'Name', 
            'Team', 
            'Age', 
            'G', 
            'PA', 
            'AB', 
            'R', 
            'H',
            '2B',
            '3B',
            'HR',
            'RBI',
            'SB',
            'CS',
            'BB',
            'SO',
            'GDP',
            'HBP',
            'SH',
            'SF',
            'IBB',
            'AVG',
            'OBP',
            'SLG',
            'BABIP',
            'ISO',
            'OPS',
            'wRC',
            'wRAA',
            'wOBA',
            'wRC+',
            'WPA',
            'WAR']
# not the full list
batting_basic_df = batting_df[var_list_basic]
batting_basic_df = batting_basic_df.sort_values(by=['IDfg', 'Season'], ascending=True)
batting_basic_df.reset_index(drop=True, inplace=True)
batting_basic_df.columns

Index(['IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'PA', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'GDP', 'HBP', 'SH',
       'SF', 'IBB', 'AVG', 'OBP', 'SLG', 'BABIP', 'ISO', 'OPS', 'wRC', 'wRAA',
       'wOBA', 'wRC+', 'WPA', 'WAR'],
      dtype='object')

## Step 5: Merge salary and batting data for analysis

In [27]:
# re_index batting_basic_df
batting_basic_df.reset_index(drop=True, inplace=True)
batting_basic_df = batting_basic_df.set_index(['IDfg'])
batting_basic_df.sort_index(level=['IDfg'], inplace=True)

In [28]:
# aggregate batting_basic based on FA year
agg_method = {'Age':'max',
             'G':'sum',
             'PA':'sum',
             'AB':'sum',
             'R':'sum',
             'H':'sum',
             '2B':'sum',
             '3B':'sum',
             'HR':'sum',
             'RBI':'sum',
             'SB':'sum',
             'CS':'sum',
             'BB':'sum',
             'SO':'sum',
             'GDP':'sum',
             'HBP':'sum',
             'SH':'sum',
             'SF':'sum',
             'IBB':'sum',
             'AVG':'mean',
             'OBP':'mean',
             'SLG':'mean',
             'BABIP':'mean',
             'ISO':'mean',
             'OPS':'mean',
             'wRC':'sum',
             'wRAA':'sum',
             'wRC+':'sum',
             'WPA':'sum',
             'WAR':'sum'}
batting_aggDF = pd.DataFrame()
for row in salaryFinalDF.itertuples():
    selected_years = [row.year_fa-1, row.year_fa-2, row.year_fa-3, row.year_fa-4, row.year_fa-5,]
    player = batting_basic_df.loc[row.IDfg]
    player = player[player['Season'].isin(selected_years)].groupby(by=['IDfg','Name']).agg(agg_method)
    player['Year_FA']=row.year_fa
    player['Salary']=row.avg_salary
    batting_aggDF = batting_aggDF.append(player)

In [29]:
# natural log salaries
batting_aggDF.insert(len(batting_aggDF.columns), 'Salary_log',
         np.log(batting_aggDF['Salary']))

In [30]:
batting_aggDF

Age    G    PA    AB    R    H   2B  3B   HR  RBI  \
IDfg  Name                                                                     
5677  A.J. Ellis            35  468  1713  1448  143  341   63   2   35  172   
746   A.J. Pierzynski       36  664  2587  2424  254  682  122   7   74  300   
      A.J. Pierzynski       35  664  2628  2455  272  695  129   7   70  290   
      A.J. Pierzynski       37  628  2414  2258  222  616  112   7   66  288   
      A.J. Pierzynski       38  613  2347  2191  217  610  107   8   66  281   
...                        ...  ...   ...   ...  ...  ...  ...  ..  ...  ...   
11368 Yasmani Grandal       29  638  2326  2000  254  471   95   4  104  294   
      Yasmani Grandal       30  663  2515  2136  286  512  102   5  117  322   
2530  Yonder Alonso         30  582  2118  1888  235  500  104   2   53  226   
8585  Yuniesky Betancourt   30  647  2498  2356  238  604  126  15   49  282   
2616  Zack Cozart           31  594  2390  2169  297  553  110  18   65  242   

                           SB  CS   BB   SO  GDP  HBP  SH  SF  IBB     AVG  \
IDfg  Name                                                                   
5677  A.J. Ellis            2   3  214  311   52   17  19  15   20  0.2306   
746   A.J. Pierzynski       5   6  101  278   76   29  12  21   21  0.2814   
      A.J. Pierzynski       5   5  109  273   76   28  15  21   24  0.2832   
      A.J. Pierzynski       4   6   91  280   71   33  10  22   17  0.2716   
      A.J. Pierzynski       1   4   95  278   73   34   4  23   17  0.2776   
...                        ..  ..  ...  ...  ...  ...  ..  ..  ...     ...   
11368 Yasmani Grandal       6   6  299  577   56    9   2  16    4  0.2350   
      Yasmani Grandal       8   7  350  601   65   12   2  15    5  0.2392   
2530  Yonder Alonso        19   7  204  323   54    9   0  17   16  0.2644   
8585  Yuniesky Betancourt  13  15   86  238   79    5  22  29    4  0.2526   
2616  Zack Cozart          17   4  164  372   49   16  20  21    9  0.2564   

                              OBP     SLG   BABIP     ISO     OPS  wRC  wRAA  \
IDfg  Name                                                                     
5677  A.J. Ellis           0.3340  0.3466  0.2680  0.1164  0.6804  183  -8.4   
746   A.J. Pierzynski      0.3154  0.4288  0.2898  0.1476  0.7440  301   6.0   
      A.J. Pierzynski      0.3184  0.4270  0.2922  0.1440  0.7452  305   0.0   
      A.J. Pierzynski      0.3068  0.4112  0.2840  0.1398  0.7180  268  -2.4   
      A.J. Pierzynski      0.3146  0.4196  0.2904  0.1422  0.7342  274  12.1   
...                           ...     ...     ...     ...     ...  ...   ...   
11368 Yasmani Grandal      0.3352  0.4412  0.2742  0.2060  0.7764  309  41.5   
      Yasmani Grandal      0.3458  0.4546  0.2746  0.2154  0.8004  359  58.9   
2530  Yonder Alonso        0.3336  0.4028  0.2912  0.1384  0.7366  255  12.3   
8585  Yuniesky Betancourt  0.2778  0.3858  0.2594  0.1332  0.6632  224 -67.4   
2616  Zack Cozart          0.3110  0.4226  0.2768  0.1660  0.7334  267  -4.9   

                           wRC+   WPA   WAR  Year_FA      Salary  Salary_log  
IDfg  Name                                                                    
5677  A.J. Ellis            464 -1.46   2.6     2017   2500000.0   14.731801  
746   A.J. Pierzynski       473 -2.58   8.8     2014    500000.0   13.122363  
      A.J. Pierzynski       468 -2.85   9.7     2013   7500000.0   15.830414  
      A.J. Pierzynski       452 -4.54   5.3     2015   2000000.0   14.508658  
      A.J. Pierzynski       483 -2.46   5.1     2016   3000000.0   14.914123  
...                         ...   ...   ...      ...         ...         ...  
11368 Yasmani Grandal       574 -1.00  22.3     2019  18250000.0   16.719676  
      Yasmani Grandal       585  1.48  25.0     2020  18250000.0   16.719676  
2530  Yonder Alonso         523  3.64   4.4     2018   8000000.0   15.894952  
8585  Yuniesky Betancourt   374 -7.71  -2.8     2013    90000